<img src="./intro_images/HDSbanner.jpg" width="100%" align="left" />

<table style="float:right;">
    <tr>
        <td>                      
            <div style="text-align: right"><a href="https://alandavies.netlify.com" target="_blank">Dr Alan Davies</a></div>
            <div style="text-align: right">Senior Lecturer health data science</div>
            <div style="text-align: right">University of Manchester</div>
         </td>
         <td>
             <img src="./intro_images/alan.png" width="30%" />
         </td>
     </tr>
</table>

# 2.0 Data types and database schema
****

#### About this Notebook
This notebook introduces the different data types and relationships between tables in a relational database.

<div class="alert alert-block alert-warning"><b>Learning Objectives:</b> 
<br/> At the end of this notebook you will be able to:
    
- Investigate how we represent relationships between tables in a database system

- Explore how we can model data in a SQL database

</div> 

<a id="top"></a>

<b>Table of contents</b><br>

2.1 [Database schema](#schema)

2.2 [Relationships](#rel)

2.3 [Dealing with dates and times](#dates)

2.4 [Constraints](#constraints)

In the last notebook we saw how we could update or delete records from a table by selecting the name of the patient we wanted to edit or remove. We also saw how this could be very problematic if 2 or more patients were to have the same name, as these changes would apply to all people who shared the same name. The way we overcome this in relational databases is to give each record a unique id, called a <code>primary key</code>. As well as this we can specify for each field what type of data we expect it to contain. Let’s see this in action as we recreate the previous <code>med_data</code> table with these new features.  

In [ ]:
%load_ext sql
%sql sqlite://

In [ ]:
%%sql
DROP TABLE IF EXISTS med_data;
CREATE TABLE med_data (
    ID INTEGER NOT NULL PRIMARY KEY,
    Name VARCHAR(255),
    Age INTEGER,
    Sex CHAR,
    "Blood pressure" CHAR(7),
    "Heart rate" INTEGER
);

<div class="alert alert-success">
<b>Note:</b> We could write the above <code>CREATE TABLE</code> code on a single line as we did previously. Here we write each field on a new line to make it easier to read.
</div>

Here, rather than just list the column names, we are stating what data type we would like to associate the data in that column with. Note that we add a <code>primary key</code> to uniquely identify each record. Data types here are similar to those we looked at in Python. An <code>INTEGER</code> is an integer (whole number) value. The <code>VARCHAR</code> is for a variable amount of text (with the maximum number of characters allowed in brackets), <code>CHAR</code> for a single character (or a set number of characters). Other versions of SQL such as MySQL have many more data types available. We can use the following code to check the data types:

In [ ]:
%%sql
PRAGMA table_info(med_data);

This shows us the field name, data type, and if the field can be NULL or not (we said the primary key cannot be NULL). We can also see any default values that are set and if a field is a primary key (pk) or not. Here we can see the field <code>ID</code> is indeed a primary key (indicated by the number 1 in column <code>pk</code>).

<div class="alert alert-success">
<b>Note:</b> In SQL <code>NULL</code> is a keyword indicating that something has no value. 
</div>

To see an exhaustive list of the data types on offer in the different versions of SQL, visit this link: <a href="https://www.w3schools.com/sql/sql_datatypes.asp" target="_blank">SQL data types</a>. Please note that because <code>SQLite</code> is a lightweight version of SQL, not all these data types are available.

<div class="alert alert-block alert-info">
<b>Task 1:</b>
<br> 
What sort of variable type would you use for a telephone number?
</div>

Although you could technically store it as an integer, or other numerical data type, this sort of data would be typically stored as a string (text). This is because you would not normally perform operations (i.e. maths) on something like a phone number. For example you are unlikely to want to compute the average phone number, as this makes no sense.

Let us add some data to the specified columns. It is always a good idea to specifically name the columns that you want the data to go into to avoid mistakes, such as putting data in the incorrect column.

In [ ]:
%%sql
INSERT INTO med_data (Name, Age, Sex, "Blood pressure", "Heart rate") VALUES("Alan Smith", 24, "M", "120/70", 78);
INSERT INTO med_data (Name, Age, Sex, "Blood pressure", "Heart rate") VALUES("Maureen Gdiver", 87, "F", "156/82", 82);
INSERT INTO med_data (Name, Age, Sex, "Blood pressure", "Heart rate") VALUES("Adam Blythe", 54, "M", "132/73", 72);
INSERT INTO med_data (Name, Age, Sex, "Blood pressure", "Heart rate") VALUES("Darren Sanders", 34, "M", "155/67", 67);
INSERT INTO med_data (Name, Age, Sex, "Blood pressure", "Heart rate") VALUES("Sally-Ann Joyce", 19, "F", "121/72", 65);

In [ ]:
%%sql
SELECT * FROM med_data;

Notice that the <code>ID</code> column has automatically incremented (been added to) for each record. This provides a unique reference for each record. This way we can change records based on this number, rather than something like name, which may not be unique and thus affect more than one record.

<a id="schema"></a>
#### 2.1 Database schema  

Each database has a <code>schema</code>. This represents the structure of the whole database. It shows the relationships between database entities (i.e. tables) and defines any constraints placed on the data. This can also be represented as a diagram. Below is an image from <a href="https://dbdiagram.io/d" target="_blank">dbdiagram.io</a> that gives an example of a database with 6 tables. It shows how they are related to each other and by which fields.

<img src="./intro_images/schema.png" width="800" />

<a id="rel"></a>
#### 2.2 Relationships

There are three types of relationship:<br />
<ul>
    <li><strong>One-to-one:</strong> seen on the diagram as a number one (<code>1</code>) alone on the line connecting fields. A row in a table can have a single matching row in another table (and vise versa).</li>
    <li><strong>One-to-Many/Many-to-One:</strong> <code>1</code> with a <code>&ast;</code> on the line. A row in one table can have many matching rows in another table. This is the most common type of relationship.</li>
    <li><strong>Many-to-many</strong> <code>&ast;</code> to a <code>&ast;</code> on the diagram. As the names suggests rows have multiple matching rows in both tables.</li>
</ul>

A primary key linked to in another table is called a <code>foreign key</code>. We will look more at relationships later. Let's make another table in our database to store details of our patients' medication use. We will call this table <code>drug_table</code>. This will include the name of the medication they are taking, the route of administration (e.g. oral), how many times a day they take it and the dose of the medication. We will set up a relationship (one-to-many) between the <code>med_data</code> and the <code>drug_table</code>.  

<div class="alert alert-block alert-info">
<b>Task 2:</b>
<br> 
Why do you think a <code>one-to-many</code> relationship is suitable here?
</div>

Because each patient can have more than one medication prescribed, so a one-to-many/many-to-one relationship is the most suitable type to use in this situation.

In [ ]:
%%sql
DROP TABLE IF EXISTS drug_table;
CREATE TABLE drug_table (
    ID INTEGER NOT NULL PRIMARY KEY,
    medication VARCHAR(255),
    route VARCHAR(4), 
    "freq per day" INTEGER,
    dose VARCHAR(255),
    patient_id INTEGER,
    FOREIGN KEY(patient_id) REFERENCES med_data(ID)
);

Note we have created an id for the patient to link the patient to the medication they take. We called this <code>patient_id</code> and specified the relationship to reference the <code>ID</code> field of the <code>med_data</code> table. Now lets put some data in the table.

In [ ]:
%%sql
INSERT INTO drug_table (medication, route, "freq per day", dose, patient_id) VALUES("AMOXICILLIN", "PO", 3, "500mg", 1);
INSERT INTO drug_table (medication, route, "freq per day", dose, patient_id) VALUES("IRBESARTAN", "PO", 1, "150mg", 2);
INSERT INTO drug_table (medication, route, "freq per day", dose, patient_id) VALUES("DIGOXIN", "PO", 1, "1.5mg", 2);
INSERT INTO drug_table (medication, route, "freq per day", dose, patient_id) VALUES("SIMVASTATIN", "PO", 1, "40mg", 3);
INSERT INTO drug_table (medication, route, "freq per day", dose, patient_id) VALUES("RAMIPRIL", "PO", 1, "2.5mg", 4);
INSERT INTO drug_table (medication, route, "freq per day", dose, patient_id) VALUES("WARFARIN", "PO", 1, "variable", 4);
INSERT INTO drug_table (medication, route, "freq per day", dose, patient_id) VALUES("SENNA", "PO", 1, "15mg", 4);
INSERT INTO drug_table (medication, route, "freq per day", dose, patient_id) VALUES("None", "NA", 0, "NA", 5);
SELECT * FROM drug_table;

This is what the relationship looks like visually. We can see that one patient (<code>1</code>) can have many (<code>&ast;</code>) medications. 

<img src="./intro_images/relate.png" width="800" />

<div class="alert alert-block alert-info">
<b>Task 3:</b>
<br> 
Go to <a href="https://dbdiagram.io/d" target="_blank">dbdiagram.io</a> and delete the code from the left hand pane. Add the code in the image below.
<img src="./intro_images/io.png" width="800" />
<br />
Now try and add another table called <code>PMH</code> (Past Medical History) with a field called <code>history</code>. Create a <code>one-to-many</code> relationship between <code>med_data</code> and the new table. <strong>HINT:</strong> You just need to make a modified version of the <code>drug_table</code> table. 
</div>

<code>
Table PMH {
  ID int
  history varchar(255)
  patient_id int [ref: > med_data.ID]
}
</code>

<img src="./intro_images/answer.PNG" width="800" />

Tools like this can be used to model our data before we spend time implementing a database. They can be useful for communication (i.e. with developers or stakeholders). They can also save a lot of wasted time by identifying logic issues, repetition of data and so on before costly development takes place.

<div class="alert alert-success">
<b>Note:</b> We recommend always modelling databases before constructing them. This will save you a lot of time and effort in the long run.  
</div>

<a id="dates"></a>
#### 2.3 Dealing with dates and times

Unlike some other versions of SQL, SQLite doesn't have any inbuilt date or time storage class. To get around this, it uses some inbuilt functions to overcome this limitation. We will look more at using functions in SQL in a later notebook. To create a table that stores the current date and time you can write something like this:

In [ ]:
%%sql
DROP TABLE IF EXISTS datetime_table;
CREATE TABLE datetime_table (
    ID INTEGER NOT NULL PRIMARY KEY,
    name VARCHAR(255),
    admission_date TEXT
);

INSERT INTO datetime_table (name, admission_date) VALUES ("Paul Davidson", datetime('now'));
SELECT * FROM datetime_table;

We can use the <code>date</code> and <code>time</code> functions to extract the individual dates and times from the admission date. Note the date is in the format YYYY-MM-DD. 

In [ ]:
%%sql
SELECT date(admission_date) FROM datetime_table;

In [ ]:
%%sql
SELECT time(admission_date) FROM datetime_table;

<div class="alert alert-success">
<b>Note:</b> Other versions of SQL (i.e. <code>MySQL</code> and <code>SQL Server</code>) support data types for storing dates and times directly. For example <code>DATE</code>, <code>DATETIME</code>, <code>TIMESTAMP</code> and <code>YEAR</code>. 
</div>

<a id="constraints"></a>
#### 2.4 Constraints

A constraint is a property given to a column that prevents the entry of inconsistent values into that field. This supports data integrity. You can also think of them as a set of rules that fields in a table must abide by. We have already seen some examples of constraints, such as: <code>PRIMARY KEY</code>, <code>FOREIGN KEY</code>, <code>UNIQUE</code> and <code>NOT NULL</code>, where a value must be present. Some other useful constraints are listed below with examples.

The <code>CHECK</code> constraint can be very useful in preventing incorrect data being added to a field. Let's say we wanted to constrain an entry for a hospital department. In this example, there are only 5 departments in the hospital. 

In [ ]:
%%sql
DROP TABLE IF EXISTS staff_table;
CREATE TABLE staff_table (
    ID INTEGER NOT NULL PRIMARY KEY,
    staff_name VARCHAR(255),
    role VARCHAR(255), 
    dept_number INTEGER
);

INSERT INTO staff_table (staff_name, role, dept_number) VALUES("Paul Smith", "Consultant oncologist", 72);
SELECT * FROM staff_table;

Here we were able to enter a department number of 72 when there should only be 5 departments. We can use <code>CHECK</code> to ensure the integrity of our data.

In [ ]:
%%sql
DROP TABLE IF EXISTS staff_table;
CREATE TABLE staff_table (
    ID INTEGER NOT NULL PRIMARY KEY,
    staff_name VARCHAR(255),
    role VARCHAR(255), 
    dept_number INTEGER CHECK (dept_number > 0 AND dept_number < 6)
);

INSERT INTO staff_table (staff_name, role, dept_number) VALUES("Paul Smith", "Consultant oncologist", 7);
SELECT * FROM staff_table;

If you scroll to the bottom of the error message you can see it mentions an integrity error <code>IntegrityError: (sqlite3.IntegrityError) CHECK constraint failed: staff_table</code>. Try changing the number to one between 1 and 5 and run the cell above again. 

We get a Python error because we are running the database in a Python notebook. Below is what the same error looks like if I try to enter department number 72 into the database using the DB Browser for SQLite tool. This shows an error message popup stating: <code>Error changing data: CHECK constraint failed: staff_table</code>.

<img src="./intro_images/check.png" width="80%" />

Another useful constraint is the <code>DEFAULT</code> constraint that can be used to provide a default value if none are provided. Lets say we wanted to provide a tax code for our employees. We could add a default standard tax code that could be overwritten if there was some exception. For example:

In [ ]:
%%sql
DROP TABLE IF EXISTS staff_table;
CREATE TABLE staff_table (
    ID INTEGER NOT NULL PRIMARY KEY,
    staff_name VARCHAR(255),
    role VARCHAR(255), 
    dept_number INTEGER,
    tax_code VARCHAR(20) DEFAULT "1185L"
);

INSERT INTO staff_table (staff_name, role, dept_number) VALUES("Paul Smith", "Consultant oncologist", 4);
SELECT * FROM staff_table;

Here we provide a standard tax code for our employees. But we can still override this if needed, say if we had a staff member on an emergency tax code:

In [ ]:
%%sql
INSERT INTO staff_table (staff_name, role, dept_number, tax_code) VALUES("Ian Ducken", "Consultant ENT", 2, "1150 W1");
SELECT * FROM staff_table;

<div class="alert alert-block alert-info">
<b>Task 4:</b>
<br> 
Create a new table called <code>nurse_band5</code>. At the time of writing this guide the salary for a band 5 (staff nurse) in the UK's NHS was between <strong>£24,907</strong> and <strong>£30,615</strong>. Make a constraint on the table so that salaries entered into the table in the field <code>salary</code> must be within this pay range. Also include a field called <code>staff_name</code> and one called <code>work_area</code> that <strong>cannot</strong> be empty, showing which ward or department the nurse works in. Finally add some data to the table to see if violating the constraints causes an error. 
</div>

In [ ]:
%%sql
DROP TABLE IF EXISTS nurse_band5;
CREATE TABLE nurse_band5 (
    ID INTEGER NOT NULL PRIMARY KEY,
    staff_name VARCHAR(255),
    work_area VARCHAR(255) NOT NULL,
    salary INTEGER CHECK (salary >= 24907 AND salary <= 30615)
);

In [ ]:
%%sql
INSERT INTO nurse_band5 (staff_name, work_area, salary) VALUES("Pauline Fowler", "Ward 12", 37000);
SELECT * FROM nurse_band5;

In [ ]:
%%sql # type in your code below


In [ ]:
%%sql # type in your code below


### Notebook details
<br>
<i>Notebook created by <strong>Dr. Alan Davies</strong>

Publish date: March 2021<br>
Review date: March 2022</i>

Please give your feedback using the button below:

<a class="typeform-share button" href="https://hub11.typeform.com/to/SsJEq1tQ" data-mode="popup" style="display:inline-block;text-decoration:none;background-color:#3A7685;color:white;cursor:pointer;font-family:Helvetica,Arial,sans-serif;font-size:18px;line-height:45px;text-align:center;margin:0;height:45px;padding:0px 30px;border-radius:22px;max-width:100%;white-space:nowrap;overflow:hidden;text-overflow:ellipsis;font-weight:bold;-webkit-font-smoothing:antialiased;-moz-osx-font-smoothing:grayscale;" target="_blank">Rate this notebook </a> <script> (function() { var qs,js,q,s,d=document, gi=d.getElementById, ce=d.createElement, gt=d.getElementsByTagName, id="typef_orm_share", b="https://embed.typeform.com/"; if(!gi.call(d,id)){ js=ce.call(d,"script"); js.id=id; js.src=b+"embed.js"; q=gt.call(d,"script")[0]; q.parentNode.insertBefore(js,q) } })() </script>

## Notes: